# Autobot Module Tutorials
This module is an abstraction
of all the methods and functionality achievable with the bot. To access it, one have to just do some function calls.

There are following three submodules in it:
- **Driver** - It contains all actuation methods (like: driving bot, running stepper)
- **BallDetector** - Self explanatory
- **SiloDetector** - Self explanatory

Following are the tutorials, on how to use **Autobot Module**.

## Using Driver Module
This tutorial will show how to use **Driver Class**. \
It sends actuating signals to the Autobot(eg: Drive, Run Stepper Motor, ...)

In [3]:
from Autobot.Driver import Driver

serial_port = '/dev/ttyACM0' # Set your serial port address
baud_rate = 115200  # Set the baud rate

driver_instance = Driver() # Initialise the driver object


In [ ]:
# The line below will give error if serial port is not found
driver_instance.initialiseSerial(address=serial_port, baudrate=baud_rate) # Initialise the serial communication


### Following methods Available with driver

In [4]:
driver_instance.moveForward()
driver_instance.rotClock()
driver_instance.rotAClock()

driver_instance.cameraDown()
driver_instance.cameraUp()

driver_instance.triggerGripper()
driver_instance.triggerRelease()

driver_instance.stop()
driver_instance.lowerSpeed()
driver_instance.upperSpeed()

driver_instance.setClutch(False) # On True all movements(Forward, Rotation) doesn't happen

#### Example Code:

In [ ]:
def focusMode(driverObj):
    driverObj.cameraDown()
    driverObj.lowerSpeed()
    driverObj.triggerRelease();

def searchMode(driverObj):
    driverObj.cameraUp()
    driverObj.upperSpeed()

def transitionMode(driverObj):
    driverObj.setClutch(True)

def activeMode(driverObj):
    driverObj.setClutch(False)

focusMode(driver_instance) # Set Focus Mode
searchMode(driver_instance) # Set Search Mode
transitionMode(driver_instance) # Set Transition Mode (All motion should seize, hence clutch is applied as in real world geared vehicles)
activeMode(driver_instance) # Set Active Mode

## Using Ball Detector Module

In [1]:
from Autobot.BallDetector import BallDetector
import cv2

weight_file_path = 'Resource/ball.pt' # Choose weight file for ball detection model (YOLO)
ballDetector = BallDetector(weight_file_path) # Initialise ball detector object
ballDetector.setEcho(True) # Set Logging Echo ('True':Output will appear in terminal else if 'OFF': in just FILE)

SYSTEM_CAM_INDEX = 0
DROID_CAM_INDEX = 1

cap = cv2.VideoCapture(SYSTEM_CAM_INDEX) # Set opencv video feed

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640) # Set width of the input image to 640 px
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480) # Set height of the input image to 480 px

def main():
    while True:
        ret, frame = cap.read() # Read the frame
        if not ret:
                print("Error: Couldn't capture a frame.")
                continue;
    
        count, pos, frame = ballDetector.getPrediction(frame) # Get prediction data from the frame
        # Note that the pos contains the position of most confident nearest ball
        if (count==0):
            ballDetector.logMessage("No Ball Detected")
            message = ballDetector.classifyMissingBall() # Classify missing condition based on last stored position
            ballDetector.logMessage(message) # Log the message to terminal and log file
        else:
            message = ballDetector.classifyBallPresence(pos[0], pos[1]) # Classify the current position of most confident ball
            ballDetector.logMessage(message)
    
        cv2.imshow("Feed", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    

### Following Methods are Available with Ball Detector Object
> You can read the details of the methods from the code itself. I have tried to be descriptive enough.\
__Autobot/BallDetector.py, Autobot/SiloDetector.py__

In [2]:
frame = None
ballDetector.getPrediction(frame)
ballDetector.classifyMissingBall()
ballDetector.classifyBallPresence(x=1, y=450)
ballDetector.eraseMemory()
ballDetector.setEcho(False)
ballDetector.logMessage(message="Vhagar is Largest dragon in the World")
ballDetector.focusAligned(x=100, y=200)

WARNING ⚠️ 'source' is missing. Using 'source=/home/sahil/Documents/ML/lib/python3.10/site-packages/ultralytics/assets'.

image 1/2 /home/sahil/Documents/ML/lib/python3.10/site-packages/ultralytics/assets/bus.jpg: 640x480 (no detections), 219.4ms
image 2/2 /home/sahil/Documents/ML/lib/python3.10/site-packages/ultralytics/assets/zidane.jpg: 384x640 (no detections), 180.6ms
Speed: 5.1ms preprocess, 200.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


2024-05-18 20:38:01 - autobot_logger - INFO - Vhagar is Largest dragon in the World


False

#### Example Code:

In [6]:
def catchBallIfInRange(ballDetectorObj, frame): # Example function to grab ball when it is in range
    count, pos, frame = ballDetectorObj.getPrediction(frame)
    if (count):
        if (ballDetectorObj.classifyBallPresence(pos[0], pos[1])=="CENTRE"):
            if (ballDetectorObj.focusAligned(pos[0], pos[1])):
                driver_instance.setClutch(True)
                driver_instance.triggerGripper()
                driver_instance.cameraUp()
                

## Using Silo Detector Module

In [1]:
from Autobot.SiloDetector import SiloDetector
import cv2

weight_file_path = 'Resource/silo.pt' # Choose weight file for Silo Detection Model (YOLO)
siloDetector = SiloDetector(weight_file_path) # Initialise Silo Detector Object
siloDetector.setEcho(True) # Set Logging Echo ('True':Output will appear in terminal else if 'OFF': in just FILE)

SYSTEM_CAM_INDEX = 0
DROID_CAM_INDEX = 1

cap = cv2.VideoCapture(SYSTEM_CAM_INDEX) # Set opencv video feed

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640) # Set width of the input image to 640 px
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480) # Set height of the input image to 480 px

def main():
    while True:
        ret, frame = cap.read() # Read the frame
        if not ret:
            print("Error: Couldn't capture a frame.")
            continue;
        
        coord = siloDetector.getLocOptimalSilo(frame) # It returns the image coordinate of centre of best silo
        
        if (coord):
            cx, cy = coord
            siloDetector.logMessage(f"({cx}, {cy})")
        
        cv2.imshow("Feed", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

> You can read more about it in code itself. As I have said its very descriptive and should be easy to understand.

## Task

Try to Implement test.py with the new Autobot Module using its methods.